In [1]:
import os
import random
import numpy as np
import tensorflow as tf

def seed_everything(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

In [2]:
import pandas as pd

In [ ]:
#Movies

In [3]:
movies = pd.read_csv(
    '/Users/taolueyang/Python_Project/recsys-rl-master/ml-1m/movies.dat',
    sep = "::",
    names = ['MovieID', 'Title', 'Genres'],
    encoding='latin-1',
    engine='python',
)


In [4]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
def extract_year(title: str) -> int:
    year = title.split(')')[-2].split('(')[-1]
    year = int(year)
    return year

movies['Year'] = movies['Title'].apply(lambda title : extract_year(title))
movies = movies.drop('Title', axis=1)

movies.head()

,MovieID,Genres,Year
0,1,Animation|Children's|Comedy,1995
1,2,Adventure|Children's|Fantasy,1995
2,3,Comedy|Romance,1995
3,4,Comedy|Drama,1995
4,5,Comedy,1995


In [6]:
genre_set = set()
for genre_str in movies['Genres']:
    for genre in genre_str.split('|'):
        genre_set.add(genre)

for genre in sorted(genre_set):
    print(genre)

Action
Adventure
Animation
Children's
Comedy
Crime
Documentary
Drama
Fantasy
Film-Noir
Horror
Musical
Mystery
Romance
Sci-Fi
Thriller
War
Western


In [7]:
def is_genre_x(genres_str: str, genre_x: str) -> int:
    if genre in genres_str:
        return 1
    return 0

for genre in genre_set:
    movies[genre] = movies['Genres'].apply(lambda genres_str: is_genre_x(genres_str, genre))

movies = movies.drop('Genres', axis=1)

movies.head()


,MovieID,Year,Adventure,Documentary,Action,Comedy,Horror,Film-Noir,Thriller,Sci-Fi,Mystery,Animation,Crime,Drama,Children's,Musical,Fantasy,Western,War,Romance
0,1,1995,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0
1,2,1995,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
2,3,1995,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,4,1995,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,5,1995,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#User 

In [8]:
users = pd.read_csv(
    '/Users/taolueyang/Python_Project/recsys-rl-master/ml-1m/users.dat',
    sep = "::",
    names = ['User_ID', 'Gender', 'Age', 'Occupation', 'Zip-code'],
    engine='python',
)

users.head()

,User_ID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [9]:
users = users.drop('Zip-code', axis=1)

users.head()

,User_ID,Gender,Age,Occupation
0,1,F,1,10
1,2,M,56,16
2,3,M,25,15
3,4,M,45,7
4,5,M,25,20


In [10]:
users['Gender'] = users['Gender'].apply(lambda gender : 1 if gender == 'F' else 0)

users.head()

,User_ID,Gender,Age,Occupation
0,1,1,1,10
1,2,0,56,16
2,3,0,25,15
3,4,0,45,7
4,5,0,25,20


In [ ]:
#Rating

In [11]:
ratings = pd.read_csv(
    '/Users/taolueyang/Python_Project/recsys-rl-master/ml-1m/ratings.dat',
    sep = "::",
    names = ['User_ID', 'MovieID', 'Rating', 'Timestamp'],
    engine='python',
)

ratings.head()

,User_ID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [12]:
ratings = ratings.drop('Timestamp', axis=1)

In [13]:
merge_df = pd.merge(users, ratings, on='User_ID')
merge_df = pd.merge(merge_df, movies, on='MovieID')

merge_df = merge_df.drop('User_ID', axis=1)
merge_df = merge_df.drop('MovieID', axis=1)

merge_df.head()

,Gender,Age,Occupation,Rating,Year,Adventure,Documentary,Action,Comedy,Horror,...,Mystery,Animation,Crime,Drama,Children's,Musical,Fantasy,Western,War,Romance
0,1,1,10,5,1975,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,56,16,5,1975,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,25,12,4,1975,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,25,7,4,1975,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,50,1,5,1975,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [16]:
target = merge_df.pop('Rating')
target = target.apply(lambda rating : rating / 5)

In [17]:
train_normalizer = layers.Normalization(axis=-1)
train_normalizer.adapt(merge_df)

Metal device set to: Apple M1 Pro


2023-12-14 20:26:32.962469: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [18]:
train_normalizer(merge_df.iloc[:3])

<tf.Tensor: shape=(3, 22), dtype=float32, numpy=
array([[ 1.7489141 , -2.4454198 ,  0.30063236, -0.81416255, -0.39325026,
        -0.08928066, -0.5887567 , -0.7443464 , -0.2875565 , -0.13636987,
        -0.48375908, -0.4319875 , -0.20457192, -0.21270467, -0.29393315,
         1.3495703 , -0.278909  , -0.2081442 , -0.19406451, -0.14531331,
        -0.27120754, -0.415944  ],
       [-0.57178915,  2.234667  ,  1.2192812 , -0.81416255, -0.39325026,
        -0.08928066, -0.5887567 , -0.7443464 , -0.2875565 , -0.13636987,
        -0.48375908, -0.4319875 , -0.20457192, -0.21270467, -0.29393315,
         1.3495703 , -0.278909  , -0.2081442 , -0.19406451, -0.14531331,
        -0.27120754, -0.415944  ],
       [-0.57178915, -0.4032001 ,  0.60684866, -0.81416255, -0.39325026,
        -0.08928066, -0.5887567 , -0.7443464 , -0.2875565 , -0.13636987,
        -0.48375908, -0.4319875 , -0.20457192, -0.21270467, -0.29393315,
         1.3495703 , -0.278909  , -0.2081442 , -0.19406451, -0.14531331,
     

In [19]:
train_x = merge_df
train_y = target

In [20]:
train_y

0          1.0
1          1.0
2          0.8
3          0.8
4          1.0
          ... 
1000204    1.0
1000205    0.6
1000206    0.2
1000207    1.0
1000208    0.8
Name: Rating, Length: 1000209, dtype: float64

In [21]:
class RecommenderNet(keras.Model):
    def __init__(
        self,
        normalizer,
        h_dim=32,
    ) -> None:
        super().__init__()
        self.normalizer = normalizer
        self.dense_1 = layers.Dense(2 * h_dim, activation=tf.nn.relu)
        self.dense_2 = layers.Dense(h_dim, activation=tf.nn.relu)
        self.dense_3 = layers.Dense(5, activation=tf.nn.softmax)

    def call(self, x):
        self.normalizer(x)
        x = self.dense_1(x)
        x = self.dense_2(x)
        x = self.dense_3(x)
        return x

In [22]:
from tensorflow.keras import callbacks

In [23]:
seed_everything(1234)

model = RecommenderNet(
    normalizer=train_normalizer,
    h_dim=64,
)

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-5,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True,
)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='mse',
)

early_stopping = callbacks.EarlyStopping(patience=2)

model.fit(
    train_x,
    train_y,
    epochs=10,
    batch_size=128,
    validation_split=0.2,
    callbacks=[
        early_stopping,
    ],
)

Epoch 1/10
6252/6252 [==============================] - 49s 7ms/step - loss: 0.4888 - val_loss: 0.4273
Epoch 2/10
6252/6252 [==============================] - 44s 7ms/step - loss: 0.4888 - val_loss: 0.4273
Epoch 3/10
6252/6252 [==============================] - 45s 7ms/step - loss: 0.4888 - val_loss: 0.4273


In [24]:
test_x = train_normalizer(merge_df.iloc[:5])
test_y = target[:5]

In [25]:
test_x

<tf.Tensor: shape=(5, 22), dtype=float32, numpy=
array([[ 1.7489141 , -2.4454198 ,  0.30063236, -0.81416255, -0.39325026,
        -0.08928066, -0.5887567 , -0.7443464 , -0.2875565 , -0.13636987,
        -0.48375908, -0.4319875 , -0.20457192, -0.21270467, -0.29393315,
         1.3495703 , -0.278909  , -0.2081442 , -0.19406451, -0.14531331,
        -0.27120754, -0.415944  ],
       [-0.57178915,  2.234667  ,  1.2192812 , -0.81416255, -0.39325026,
        -0.08928066, -0.5887567 , -0.7443464 , -0.2875565 , -0.13636987,
        -0.48375908, -0.4319875 , -0.20457192, -0.21270467, -0.29393315,
         1.3495703 , -0.278909  , -0.2081442 , -0.19406451, -0.14531331,
        -0.27120754, -0.415944  ],
       [-0.57178915, -0.4032001 ,  0.60684866, -0.81416255, -0.39325026,
        -0.08928066, -0.5887567 , -0.7443464 , -0.2875565 , -0.13636987,
        -0.48375908, -0.4319875 , -0.20457192, -0.21270467, -0.29393315,
         1.3495703 , -0.278909  , -0.2081442 , -0.19406451, -0.14531331,
     

In [26]:
test_y

0    1.0
1    1.0
2    0.8
3    0.8
4    1.0
Name: Rating, dtype: float64

In [27]:
model(test_x)

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[0.16419932, 0.17616944, 0.20921862, 0.20338163, 0.24703099],
       [0.09824777, 0.18692377, 0.11844304, 0.17504507, 0.42134035],
       [0.15127721, 0.20234483, 0.18628351, 0.17266828, 0.28742614],
       [0.15698677, 0.19437441, 0.20610961, 0.17045824, 0.27207094],
       [0.12909497, 0.15808721, 0.19051899, 0.17956185, 0.34273696]],
      dtype=float32)>